In [6]:
library(repr)
library(tidyverse)
library(tidymodels)
library(readr)

#### Group project proposal

# Data Analysis on Air Quality Index of Counties in the US

DSCI100 group porject-006-14  
Group member:  
`Dorthy GER`  
`Charlie Hua`  
`Callie Phelps`  
`Hancheng Zhang 29643160`


## Introduction

In [7]:
url <- "https://raw.githubusercontent.com/Hansen0014/Air-quality-data/main/annual_aqi_by_county_2021.csv"
Air_quality <- read.csv(url)

In [9]:
head(Air_quality)

,State,County,Year,Days.with.AQI,Good.Days,Moderate.Days,Unhealthy.for.Sensitive.Groups.Days,Unhealthy.Days,Very.Unhealthy.Days,Hazardous.Days,Max.AQI,X90th.Percentile.AQI,Median.AQI,Days.CO,Days.NO2,Days.Ozone,Days.SO2,Days.PM2.5,Days.PM10
,<chr>,<chr>,<int>,<int>,<int>,<int>,<int>,<int>,<int>,<int>,<int>,<int>,<int>,<int>,<int>,<int>,<int>,<int>,<int>
1,Alabama,Baldwin,2021,203,190,13,0,0,0,0,61,49,37,0,0,166,0,37,0
2,Alabama,Clay,2021,73,66,7,0,0,0,0,67,50,28,0,0,0,0,73,0
3,Alabama,DeKalb,2021,239,211,28,0,0,0,0,84,51,38,0,0,213,0,26,0
4,Alabama,Elmore,2021,180,177,3,0,0,0,0,87,45,34,0,0,180,0,0,0
5,Alabama,Etowah,2021,204,178,26,0,0,0,0,77,54,38,0,0,158,0,46,0
6,Alabama,Jefferson,2021,182,90,90,1,0,1,0,213,74,51,2,1,65,0,113,1
